<img src="../static/logopython.png" alt="Logo Python" style="width: 300px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 300px; display: inline"/>

# Clase 5: Metaprogramación en Python

En la metaprogramación, escribimos <strong>código que trabaja con código</strong>. Algo para lo que Python 3 está <strong>especialmente bien preparado</strong>

Veamos a continuación para qué nos puede interesar escribir código que trabaja con código.

## ¿Qué es eso de la metaprogramación?

Como ya hemos dicho, la metaprogramación consiste en escribir código que va a trabajar con el código ya escrito, incluso consigo mismo.

Al ser Python un lenguaje dinámico, no podemos saber las interioridades de los componentes con los que estemos trabajando, salvo que los inspeccionemos en tiempo de ejecución. Por ejemplo

In [3]:
# Añadimos atributos a una instancia de una clase, una vez creada
class Empleado(object):
    pass

jorge = Empleado()
jorge.guapo = "Por supuesto"

print("Es Jorge guapo: {}".format(jorge.guapo))

Es Jorge guapo: Por supuesto


En la manera más básica de metaprogramación, podemos manipular los atributos de una clase, mediante el trio de funciones [hasattr](https://docs.python.org/3/library/functions.html#hasattr), [getattr](https://docs.python.org/3/library/functions.html#getattr), [setattr](https://docs.python.org/3/library/functions.html#setattr)

In [6]:
# Probando hasattr, getattr, setattr
print(hasattr(jorge, 'guapo'))
print(getattr(jorge, 'guapo'))

print(hasattr(jorge, 'listo'))
setattr(jorge, 'listo', 'Más que las monas')
print(getattr(jorge, 'listo'))

True
Por supuesto
False
Más que las monas


TODO: Meter la parte de introspección del capítulo 3 del libro. Es muy cortito y habla de docstrings, tipos, etc

## ¿Y para qué me vale esto?

La metaprogamación es especialmente útil cuando:

* Construyes herramientas para otros programadores (ej: un IDE)
* Quieres añadir ciertos _extras_ a tu código, para hacerlo más seguro o más fácilmente depurable
* Quieres aprender cómo funciona realmente el lenguaje por dentro... y manipularlo
* ...

Es posible que estos argumentos no te parezcan suficientes. No hay problema. [Tim Peters](http://c2.com/cgi/wiki?TimPeters) dijo esto acerca de las metaclases (uno de los componentes fundamentales de la metaprogramación):

_[Metaclasses] are deeper magic than 99% of
users should ever worry about. If you
wonder whether you need them, you
don't (the people who actually need them
know with certainty that they need them,
and don't need an explanation about why)._

Así que, si no te ha surgido la necesidad de usar metaprogramación, no te preocupes ;-). Nosotros, al menos, vamos a ver un ejemplo de uso

## Caso de uso: depuración de código

¿Cómo depuramos nuestro código? Una primera aproximación (pobre) es... usando print

In [7]:
# Así es como depuran los muggles
def add(x, y):
    print('Dentro de la función add')
    return x + y

add(2, 2)

Dentro de la función add


4

Precioso. ¿Y si tenemos más funciones?

In [4]:
# La cosa se complica...
def add(x, y):
    print('add')
    return x + y

def sub(x, y):
    print('sub')
    return x - y

def mul(x, y):
    print('mul')
    return x * y

def div(x, y):
    print('div')
    return x / y

add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

add
sub
mul
div


8.0

In [1]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../static/styles/style.css'
HTML(open(css_file, "r").read())